In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import multiprocessing
#from unidecode import unidecode
from textblob import TextBlob

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install emot

     |████████████████████████████████| 61 kB 16 kB/s 


In [ ]:
DF = pd.read_csv('/content/drive/MyDrive/NLP/Scala_test_data.csv')
DF.head()

,Content_Id,Article_Title_Txt,Locale_Cd,comment_ts,Rating_Comment_Txt
0,HT204168,NaN,en_US,17-01-18 20:19,I cant accept FaceTime calls
1,HT204053,NaN,en_US,29-07-17 16:19,genius bar appointment
2,HT207043,NaN,en_US,18-01-18 22:07,So basically if my phone gets wet there isnt ...
3,HT204385,NaN,en_US,12-02-18 21:10,Why do you force people to deauthorize all com...
4,BK_BF0D7D2D2C2DCA6D2A6301E487ABFD7E_AMSA9337677E,NaN,en_US,05-02-18 21:14,my device is listed in the audio device list ...


In [ ]:
#DF = DF.rename(columns={:'Text'})

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
    

In [ ]:
def make_re_compatible_pattern(emot):
    pattern =  emot.replace(")","\)").replace("]","\]").replace("(","\(").replace("[","\[") 
    return pattern

In [ ]:
make_re_compatible_pattern(":-))))")

':-\\)\\)\\)\\)'

In [ ]:
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
# Converting emojis to words
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
        return text

# the reason for this function is how re handels () [] 
def make_re_compatible_pattern(emot):
    pattern =  emot.replace(")","\)").replace("]","\]").replace("(","\(").replace("[","\[") 
    return pattern
# Converting emoticons to words    
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, EMOTICONS_EMO[emot].split(",")[0])
    return text
# Example
text = "Hello :-) "
convert_emoticons(text)

'Hello Happy face smiley '

In [ ]:
from bs4 import BeautifulSoup
#Function for removing html
def html(text):
    return BeautifulSoup(text, "lxml").text
# Examples
text = """<div>
<h1> This</h1>
<p> is</p>
<a href="https://www.abc.com/"> ABCD</a>
</div>
"""
print(html(text))
# Passing the function to 'text_rare'
#df['text_rare'] = df['text_rare'].apply(html)


 This
 is
 ABCD




In [ ]:
# Function for url's
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
# Examples
text = "This is my website, https://www.abc.com"
remove_urls(text)
#Passing the function to 'text_rare'
#df['text_rare'] = df['text_rare'].apply(remove_urls)

'This is my website, '

In [ ]:

# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)


In [ ]:
def process_text(dataF ,cl_name : str,new : str, remove_stop_words = True ):
    new_cl_name = new + cl_name
    
    
    dataF[cl_name] = dataF[cl_name].apply(lambda x: str(x))

    #remove wierd html ascii symbols
    dataF[cl_name] = dataF[cl_name].apply(lambda x: re.sub("&#[0-9]*;", "'",x))

    #remove wierd &quot;
    dataF[cl_name] = dataF[cl_name].apply(lambda x: re.sub("&[a-z]*;", "'",x))


    # make it lower case
    dataF[new_cl_name] = dataF[cl_name].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    #df[new_cl_name].head()
      
    # Converting emojis to words
    dataF[new_cl_name] = dataF[new_cl_name].apply(convert_emojis)

    # Converting emoticons to words
    dataF[new_cl_name] = dataF[new_cl_name].apply(convert_emoticons)

    # remove hashtags
    dataF[cl_name] = dataF[cl_name].apply(lambda x: re.sub("#[a-z]*", '',x))

    # remove punctuation but not .
    
    dataF[new_cl_name] = dataF[new_cl_name].str.replace(".",'SW34456gfsju9345')

    dataF[new_cl_name] = dataF[new_cl_name].str.replace('[^\w\s]',"'")

    dataF[new_cl_name] = dataF[new_cl_name].apply(expand_contractions)

    dataF[new_cl_name] = dataF[new_cl_name].str.replace('[^\w\s]'," ")

    dataF[new_cl_name] = dataF[new_cl_name].str.replace('SW34456gfsju9345','.')
    
    # remove stop words like a, the, etc... 
    if remove_stop_words:
        stop = stopwords.words('english')
        dataF[new_cl_name] = dataF[new_cl_name].apply(lambda x: [x for x in x.split() if x not in stop])

    # Spell check using text blob
    #dataF[new_cl_name] = dataF[new_cl_name].apply(lambda x: str(TextBlob(x).correct()))
    # takes too long
  

    # remove urls  
    dataF[new_cl_name] = dataF[new_cl_name].apply(remove_urls)


In [ ]:
DF = DF.rename(columns={'Rating_Comment_Txt':'Text'})

In [ ]:
process_text(DF,'Text','sw_',remove_stop_words=True)

TypeError: ignored

In [ ]:
temp_df = DF.sw_text

In [ ]:
def truncate_long_sent(DF):
    for i in range(DF.shape[0]):
        d = DF[i]
        d_l = d.split()
        if len(d_l) >= 400:
            d_l = d_l[:40]
            DF[i] = ' '.join(d_l)
#truncate_long_sent(DF.sw_Text)

In [ ]:

def preprocess_DF(PATH, Text_col :str,max_words : int):
    DF = pd.read_csv(PATH)
    DF = DF.rename(columns={Text_col : 'Text'})
    process_text(DF,'Text','sw_', remove_stop_words=False)
    DF.head()
    truncate_long_sent(DF.sw_Text)
    path_list = PATH.split('/')
    p = PATH.split('/')
    new_PATH = "/".join(p[:-1] + ['Refined_' + p[-1]])
    DF.to_csv(new_PATH)
    print(f'saved data to {new_PATH}')
    return DF

In [ ]:
DF = preprocess_DF('/content/drive/MyDrive/NLP/Final_8_combo_data.csv','Text',400)


satved data to /content/drive/MyDrive/NLP/Refined_Final_8_combo_data.csv


In [ ]:
DF.tail()

,Unnamed: 0,Text,Emotions,sw_Text
328867,3080,MT @AliHaggett: Looking forward to our public ...,happy,mt alihaggett looking forward to our public ...
328868,3081,@britishmuseum Upper arm guard?,nocode,britishmuseum upper arm guard
328869,3082,@MrStuchbery @britishmuseum Mesmerising.,happy,mrstuchbery britishmuseum mesmerising.
328870,3083,@NationalGallery The 2nd GENOCIDE against Biaf...,not-relevant,nationalgallery the 2nd genocide against bia...
328871,3084,@britishmuseum Experience from DukeofWellingt...,nocode,britishmuseum experience battlewaterloo from...


In [ ]:
lines = []
def find_word(DF,word):
    for x in DF.sw_Text:
        if len(re.findall(word,str(x))) > 0 :
            lines.append(x)
            print(x)
find_word(DF,'android')

i liked the japandroids for that one year
i m seeing the error using chrome both on macos desktop and android. i m glad it seems to just be me though. apologies for the snarkiness.
yeah i had android and was having difficulties but i was able to get it easily when using my gf s mac. thanks 
i m seeing similar on android  chrome browser. other than that  great stuff 
 samsung other android phones don t have bixby  that s samsung s bs  and yeah  it s terrible. i have the button   feature completely disabled 
imagine spending 1000  on the exact same phone every 6 months. this comment was made by the android gang
unfortunately  on the stupid official reddit mobile app it doesn t work. i miss my android and reddit is fun app
avid android user here. my xiaomi mi mix 2 is by far the sexiest and fastest phone i have ever had. such bliss when using it.
 all4 is the android app it designed to be buggy and work sporadically on a fire tv box   shocking
 all4 is the android app it designed to be bu

In [ ]:
DF = DF.drop_duplicates(subset=['sw_Text'])
DF.head()

,Unnamed: 0,Text,Emotions,sw_Text
0,0,im feeling rather rotten so im not very ambiti...,sadness,im feeling rather rotten so im not very ambiti...
1,1,im updating my blog because i feel shitty,sadness,im updating my blog because i feel shitty
2,2,i never make her separate from me because i do...,sadness,i never make her separate from me because i do...
3,3,i left with my bouquet of red and yellow tulip...,joy,i left with my bouquet of red and yellow tulip...
4,4,i was feeling a little vain when i did this one,sadness,i was feeling a little vain when i did this one


In [ ]:
DF.to_csv('/content/drive/MyDrive/NLP/Refined_Final_8_combo_data.csv')

In [ ]:
DF = pd.read_csv('/content/drive/MyDrive/NLP/Refined_Final_8_combo_data.csv')

In [ ]:
DF.Emotions.value_counts()

no emotion           68293
neutral              39047
sadness              15671
happiness            15123
joy                  13140
anger                 8889
worry                 8431
love                  7241
surprise              6950
fear                  5778
admiration            5096
approval              3260
amusement             2787
annoyance             2664
gratitude             2594
curiosity             2202
disapproval           2113
fun                   1774
relief                1633
nocode                1558
confusion             1535
non-neutral           1477
disgust               1458
hate                  1321
disappointment        1279
optimism              1220
caring                1207
happy                 1136
realization            891
excitement             844
empty                  822
enthusiasm             756
desire                 681
remorse                496
embarrassment          305
not-relevant           186
boredom                179
n

In [ ]:
emo_con_list = ['no emotion',
                'neutral',
                'sadness',
                'happiness',
                'joy',
                'anger',
                'worry',
                'love',
                'surprise',
                'fun',
                'nocode',
                'happy',
                'angry',
                'sad',
                'happy|surprise',
                'happy|sad',
                'sad|angry',
                'sad|disgust',
                'sad|disgust|angry']    

In [ ]:

emo_dict1 = {'Neutral':0,
    'worry':1,
    'sad':2,
    'frustrated':3,
    'anger':4,
    'surprise':5,
    'love':6,
    'happy':7
    }


In [ ]:
#“Happiness”, “Surprise”, “Sadness”, “Anger”

In [ ]:
#DF = DF[DF.isin(emo_con_list)]

In [ ]:
#DF.Emotions.value_counts()

In [ ]:
def make_df(DF,emo_l,new_emo):
    df = pd.DataFrame()
    for emo in emo_l:
        df = pd.concat([df,DF[DF.Emotions == emo]])
        df.Emotions = new_emo
    return df

In [ ]:
n_l = ['no emotion','neutral','nocode']
new_emo = 'neutral'
neutral_df = make_df(DF,n_l,new_emo)


In [ ]:
e_l = ['worry']
new_emo = 'worry'
worry_df = make_df(DF,e_l,new_emo)
worry_df.Emotions.isna().sum()

0

In [ ]:
e_l = ['sadness','sad','sad|angry','sad|disgust','sad|disgust|angry']
new_emo = 'sad'
sad_df = make_df(DF,e_l,new_emo)
sad_df.Emotions.isna().sum()

0

In [ ]:
e_l = ['anger','angry']
new_emo = 'anger'
anger_df = make_df(DF,e_l,new_emo)
anger_df.Emotions.isna().sum()

0

In [ ]:
e_l = ['surprise']
new_emo = 'surprise'
surprise_df = make_df(DF,e_l,new_emo)
surprise_df.Emotions.isna().sum()

0

In [ ]:
e_l = ['love']
new_emo = 'love'
love_df = make_df(DF,e_l,new_emo)
love_df.Emotions.isna().sum()

0

In [ ]:
e_l = ['happy','happiness','happy|surprise','happy|sad',]
new_emo = 'happy'
happy_df = make_df(DF,e_l,new_emo)
happy_df.Emotions.isna().sum()

0

In [ ]:
path = '/content/drive/MyDrive/NLP/Final_datasets/.csv'
path_l = path.split('/')
#df_list = [neutral_df,worry_df,sad_df,anger_df,surprise_df,love_df,happy_df]
emo_list = ['Neutral', 'worry','sad','anger','surprise','love','happy']

In [ ]:
def save_df(df_list,emo_list):
    for i in range(len(emo_list)):
        save_path  = "/".join(path_l[:-1]+[emo_list[i] + path_l[-1]])
        df_list[i].to_csv(save_path)
        

In [ ]:
import pandas as pd


In [ ]:
save_df(df_list,emo_list)

In [ ]:
df_dict = {}
for emo in emo_list:
    save_path  = "/".join(path_l[:-1]+[emo + path_l[-1]])
    df_dict[emo] = pd.read_csv(save_path)
    


108898
8431
15708
8946
6950
7241
16279


In [ ]:
def combine_dfs(df_dict,max_size):
    big_df = pd.DataFrame()
    for emo in emo_list:
        dfi = df_dict[emo]
        size = dfi.shape[0]
        if size <= max_size:
            big_df = pd.concat([big_df,dfi])
        else :
            dfi = dfi.sample(frac=1)
            dfi = dfi[:max_size]
            big_df = pd.concat([big_df,dfi])
    big_df = big_df.sample(frac = 1)
    big_df = big_df.reset_index(drop=True)
    return big_df

In [ ]:
big_df = combine_dfs(df_dict,9000)

In [ ]:
big_df = big_df.dropna()

In [ ]:
big_df = big_df.reset_index(drop=True)

In [ ]:
big_df.to_csv('/content/drive/MyDrive/NLP/Final_datasets/9kmax.csv')

In [ ]:
big_df.Emotions.value_counts()

happy       9000
neutral     9000
sad         9000
anger       8946
worry       8431
love        7241
surprise    6950
Name: Emotions, dtype: int64